In [20]:
# ✅ Environment Setup
import os
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
from langchain_openai import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent, AgentType
from langchain.memory import ConversationBufferMemory


In [21]:
# 🔐 Load API Keys
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("❌ OPENAI_API_KEY is not loaded from .env")

print("✅ API key successfully loaded")

✅ API key successfully loaded


In [22]:
csv_path = "/Users/keshavsaraogi/Desktop/indorama/eureka-data/clean-csv/cleaned_finance_packaging.csv"
df = pd.read_csv(csv_path)

In [23]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.5
)

In [24]:
finance_csv_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
finance_sql_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [25]:
# 🤖 CSV Agent
csv_agent = create_csv_agent(
    llm,
    csv_path,
    verbose=True,
    allow_dangerous_code=True,
    handle_parsing_errors=True,
    max_iterations=20,
    max_execution_time=60,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    memory=finance_csv_memory
)

/opt/anaconda3/envs/eureka-env/lib/python3.10/site-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True, 'memory': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history')} which are no longer supported.
  warnings.warn(


In [26]:
# 🛢️ SQL Agent
engine = create_engine("sqlite:///finance_packaging.db")
df.to_sql("finance_packaging_data", engine, index=False, if_exists="replace")
db = SQLDatabase(engine)

sql_agent = create_sql_agent(
    llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=20,
    max_execution_time=60,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    memory=finance_sql_memory
)


In [27]:
# 🧭 Routing Function
def route_query(query: str):
    query = query.lower()
    if any(keyword in query for keyword in ["sum", "avg", "group", "filter", "division", "unit price", "balance"]):
        return "sql"
    else:
        return "csv"

# Master Agent
def master_agent(query: str):
    source = route_query(query)
    if source == "sql":
        return sql_agent.run(query)
    else:
        return csv_agent.run(query)

In [28]:
# 🧠 Recall Last Interaction
def recall_last_interaction(memory):
    messages = memory.chat_memory.messages
    if len(messages) >= 2:
        user_msg = messages[-2].content
        ai_msg = messages[-1].content
        return f"🧠 Last Question: {user_msg}\n💬 Last Answer: {ai_msg}"
    return "🧠 No previous memory found."


In [29]:
def show_full_chat(memory):
    print("🧠 Chat History:")
    for msg in memory.chat_memory.messages:
        role = msg.type.capitalize()
        print(f"{role}: {msg.content}")

In [30]:
print(master_agent("What is the sum of the ending balance of the ending balance in global currency? Explain how you arrived at the answer."))



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


finance_packaging_data
Invoking: `sql_db_schema` with `{'table_names': 'finance_packaging_data'}`



CREATE TABLE finance_packaging_data (
	"Company ID" TEXT, 
	"Company Name" TEXT, 
	"G/L Account ID" TEXT, 
	"G/L Account Name" TEXT, 
	"Clearing Date" TEXT, 
	"Customer ID" TEXT, 
	"Customer Name" TEXT, 
	"Division" TEXT, 
	"Fixed Asset ID" TEXT, 
	"G/L Account Type" TEXT, 
	"G/L Account Type Description" TEXT, 
	"Material ID" TEXT, 
	"Material Name" TEXT, 
	"Plant ID" TEXT, 
	"Plant Name" TEXT, 
	"Supplier ID" TEXT, 
	"Supplier Name" TEXT, 
	"Cost Center ID" TEXT, 
	"Cost Center Name" TEXT, 
	"Starting Balance in Global Currency" FLOAT, 
	"Debit Balance in Global Currency" FLOAT, 
	"Credit Balance in Global Currency" FLOAT, 
	"Ending Balance in Global Currency" FLOAT
)

/*
3 rows from finance_packaging_data table:
Company ID	Company Name	G/L Account ID	G/L Account Name	Clearing Date	Customer ID	Cus

In [31]:
print(recall_last_interaction(finance_sql_memory))

🧠 No previous memory found.


In [32]:
# agent_executer.invoke("How many total company codes are there in the dataset? Name them all.")

In [33]:
# agent_executer.invoke("What is the sum of the debit balance in global currency? Explain how you arrived at the answer.")

In [34]:
# agent_executer.invoke("What is the sum of the total credit balance in global currency? Explain how you arrived at the answer.")

In [35]:
# agent_executer.invoke("Give me a list of the top 3 companies with the highest total credit balance in global currency. Explain how you arrived at the answer.")

In [36]:
# agent_executer.invoke("What is the sum of the ending balance of the ending balance in global currency? Explain how you arrived at the answer.")

In [37]:
# master_agent("What is the sum of the ending balance of the ending balance in global currency? Explain how you arrived at the answer.")

In [38]:
# master_agent("What was the question I asked last and what was your answer?")